# Blade-chest innerモデル（を雑に拡張して）Mリーガーの強さと相性を推定する
<span style="color:red">**注意**</span>：このノートブックで示されるMリーガーの強さ・相性は、タイトルにもある通り、（雑に拡張した）**Blade-chest innerモデルで推定**されたものです。
この記事で示された強さや相性は、あくまで推定したものであり、また、モデルが異なれば結果も変わる、ということにご注意ください。

# データの読み込み

In [1]:
import numpy as np
from load_dataset import remove_ranking_point, add_ranking_point, load_mleague_dataset
from blade_chest import BladeChestInner
from sklearn.model_selection import GridSearchCV

X, y = load_mleague_dataset("./data/")
y_original = remove_ranking_point(y) # 素点

print(X)
print(y)
print(y_original)
n_players = len(np.unique(X))
print(f"プレイヤ数：{n_players}")

[['瑞原明奈' '魚谷侑未' '日向藍子' '内川幸太郎']
 ['沢崎誠' '滝沢和典' '瀬戸熊直樹' '多井隆晴']
 ['滝沢和典' '多井隆晴' '園田賢' '佐々木寿人']
 ...
 ['瀬戸熊直樹' '松本吉弘' '勝又健志' '前原雄大']
 ['白鳥翔' '内川幸太郎' '小林剛' '瀬戸熊直樹']
 ['朝倉康心' '前原雄大' '多井隆晴' '村上淳']]
[[ 77.2  10.1 -26.  -61.3]
 [ 62.9  20.9 -21.8 -62. ]
 [ 61.9   5.8 -23.6 -44.1]
 ...
 [ 61.2   7.6 -18.2 -50.6]
 [ 60.4   3.4 -20.  -43.8]
 [ 56.   13.9 -23.1 -46.8]]
[[ 32.2   5.1 -11.  -26.3]
 [ 17.9  15.9  -6.8 -27. ]
 [ 16.9   0.8  -8.6  -9.1]
 ...
 [ 16.2   2.6  -3.2 -15.6]
 [ 15.4  -1.6  -5.   -8.8]
 [ 11.    8.9  -8.1 -11.8]]
プレイヤ数：30


## Blade-chest innerモデルの学習
### 順位点も込み

In [2]:
bc = BladeChestInner(n_components=5, alpha=1e-6, beta=1e-2, max_iter=1000, verbose=False)
bc.fit(X, y)
strength_sorted = np.sort(bc.coef_)[::-1]
players_sorted = bc.enc_.inverse_transform(np.argsort(bc.coef_)[::-1])

print("強さランキング")
print(players_sorted)
print("強さ")
print(strength_sorted)

comparison_matrix = np.dot(bc.V_, bc.U_.T) - np.dot(bc.U_, bc.V_.T)
indices = np.argmax(comparison_matrix)
player1 = indices // n_players
player2 = indices % n_players
print(f"最も相性差が大きいペア：{bc.enc_.inverse_transform([player1, player2])}")

強さランキング
['堀慎吾' '日向藍子' '小林剛' '黒沢咲' '佐々木寿人' '藤崎智' '茅森早香' '多井隆晴' '萩原聖人' '和久津晶'
 '鈴木たろう' '内川幸太郎' '滝沢和典' '松本吉弘' '園田賢' '勝又健志' '前原雄大' '魚谷侑未' '朝倉康心' '近藤誠一'
 '村上淳' '瀬戸熊直樹' '沢崎誠' '石橋伸洋' '白鳥翔' '高宮まり' '二階堂亜樹' '岡田紗佳' '瑞原明奈' '丸山奏子']
強さ
[ 9.17212301  4.24289557  2.89589169  1.80783425  1.77572023  1.67027154
  1.49919506  1.40084661  1.27833901  1.24378347  1.13028355  0.85872262
  0.77469368  0.46188782  0.43453182  0.15987008 -0.31528726 -0.44386633
 -0.46694414 -0.60837206 -1.03624014 -1.25918541 -1.32649404 -2.38195204
 -2.88498098 -3.1060321  -3.9135168  -4.14336367 -4.28130286 -4.6393522 ]
最も相性差が大きいペア：['和久津晶' '茅森早香']


### 素点で推定

In [3]:
bc = BladeChestInner(n_components=5, alpha=1e-6, beta=1e-2, max_iter=1000, verbose=False)
bc.fit(X, y_original)
strength_sorted = np.sort(bc.coef_)[::-1]
players_sorted = bc.enc_.inverse_transform(np.argsort(bc.coef_)[::-1])

print("強さランキング")
print(players_sorted)
print("強さ")
print(strength_sorted)

comparison_matrix = np.dot(bc.V_, bc.U_.T) - np.dot(bc.U_, bc.V_.T)
indices = np.argmax(comparison_matrix)
player1 = indices // n_players
player2 = indices % n_players
print(f"最も相性差が大きいペア：{bc.enc_.inverse_transform([player1, player2])}")

強さランキング
['堀慎吾' '多井隆晴' '内川幸太郎' '小林剛' '園田賢' '丸山奏子' '滝沢和典' '黒沢咲' '鈴木たろう' '松本吉弘'
 '佐々木寿人' '萩原聖人' '日向藍子' '魚谷侑未' '近藤誠一' '和久津晶' '藤崎智' '茅森早香' '勝又健志' '高宮まり'
 '瀬戸熊直樹' '瑞原明奈' '朝倉康心' '前原雄大' '石橋伸洋' '二階堂亜樹' '沢崎誠' '村上淳' '白鳥翔' '岡田紗佳']
強さ
[ 2.70856904  1.53950268  1.40339251  1.28347605  1.28308986  0.74333526
  0.69801791  0.68291046  0.59108214  0.45886158  0.32696652  0.22231166
  0.20641158  0.15954446  0.14631176  0.12337122 -0.06824308 -0.07144087
 -0.18800129 -0.33269427 -0.42498483 -0.53419567 -0.72436916 -0.95078518
 -1.16588839 -1.32428748 -1.34229675 -1.36447319 -1.78035465 -2.3051399 ]
最も相性差が大きいペア：['堀慎吾' '沢崎誠']
